In [39]:
import sys
sys.path.append('../src/')
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from utils.visual_functions import *
from utils.utils import *
from utils.data_generator import *
import numpy as np
from sklearn.manifold import TSNE
%matplotlib inline
fig_path="../figure/paper/"
seed = 4783957
print("set seed")
np.random.seed(seed)
import torch
from palettable.colorbrewer.qualitative import Pastel1_9
from utils.data_generator import get_data, get_loaders
plaid_names = ['CFL','ILB','Waterkettle','Fan','AC','HairIron','LaptopCharger','SolderingIron','Fridge','Vacuum',
               'CoffeeMaker','FridgeDefroster']

set seed


### Functions

In [40]:
def calculatePower(train_current, train_voltage, NN=500):
  

    n = len(current)
    PQ = np.empty([n,2])
    for i in range(n):
        # extract current and voltage in two cycle
        # extract current and voltage in two cycle
        temp_I = current[i]
        temp_V = voltage[i]
        Irms = np.mean(temp_I**2)**0.5
        Vrms = np.mean(temp_V**2)**0.5

        # Instantaneous power: obtained by multiplying the instantaneous voltage and current values
        Pinst = temp_I * temp_V
        Pactive = np.mean(Pinst) 

        # Apparent power: obtained by multiplying the RMS values of voltage and current
        Papparent = Vrms*Irms;

        # Ractive power if power triangle holds:
        Preactive = np.sqrt(Papparent**2 - Pactive**2);




        PQ[i,0]=Pactive
        PQ[i,1]=Preactive
    return PQ



In [41]:
def plot_PQPlane(labels, PQ, legends=None, sample=True):
    
    if legends:
        Legends=legends
    else:
        Legends=np.unique(labels)
        
    clrs = sns.color_palette("Paired", len(Legends))
    markers='1v^<P4ph+DX*8os3<-'
    
    minx,maxx,miny,maxy = (0,0,0,0)
    

    for i, app in enumerate(Legends):
        ix=np.where(labels==app)[0]
        if sample:
            ix1 = np.random.randint(PQ.shape[0],size=6)
            tempx = PQ[ix1,0]
            tempy = PQ[ix1,1]
        else:
            tempx = PQ[ix,0]
            tempy = PQ[ix,1]
        plt.scatter(tempx,tempy,marker=markers[i], s=30, label=Legends[i], color=clrs[i])
        minx = min(minx,min(tempx))
        maxx = max(maxx,max(tempx))
        miny = min(miny,min(tempy))
        maxy = max(maxy,max(tempy))    
    plt.axis([minx*0.9,maxx*1.1,miny*0.9,maxy*1.1])
    plt.xlabel('Real Power(W)')
    plt.ylabel('Reactive Power (VAR)')
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

    
def tsne_visualizer(labels, feature, legends=None):
    if legends:
        Legends=legends
    else:
        Legends=np.unique(labels)
    clrs = sns.color_palette("Paired", len(Legends))
    model  = TSNE(perplexity=20, min_grad_norm=1E-12, n_iter=10000, random_state=42)
    #model = TSNE(n_components=2, random_state=0, perplexity=20, n_iter=1000)
    TSNEF  = model.fit_transform(feature.reshape(len(feature), -1))
    
    for i, app in enumerate(Legends):
        ix=np.where(labels==app)[0]
        plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=app, color=clrs[i])
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    plt.xlabel("tsne-2d-one")
    plt.ylabel("tsne-2d-two")
    
def list_intersection(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    return a_set.intersection(b_set) 



### PLAID

In [42]:
current, voltage, labels, I_max = get_data(data_type="plaid")

Load plaid aggregated data from ../data/plaid/


In [43]:
for x in labels:
    if len(x)==4:
        print(x)

['AC', 'Waterkettle', 'SolderingIron', 'SolderingIron']
['AC', 'CoffeeMaker', 'SolderingIron', 'SolderingIron']
['SolderingIron', 'SolderingIron', 'AC', 'CoffeeMaker']
['SolderingIron', 'SolderingIron', 'CoffeeMaker', 'Vacuum']
['SolderingIron', 'SolderingIron', 'ILB', 'AC']
['CoffeeMaker', 'Vacuum', 'SolderingIron', 'SolderingIron']
['Fridge', 'SolderingIron', 'SolderingIron', 'ILB']
['ILB', 'SolderingIron', 'SolderingIron', 'Fridge']
['Vacuum', 'SolderingIron', 'SolderingIron', 'FridgeDefroster']


In [44]:
active_apps = np.array([len(set(x)) for x in labels])
l, count=np.unique(active_apps, return_counts=True)
volgorde = np.argsort(count)
fig = figure(fig_width=6)
barwidth = 0.3
plt.bar(np.array(l), count, width=barwidth, color="C1")
plt.xticks(np.array(l), np.array(["1", '2', '3', '4']));
plt.tight_layout()
ax = plt.gca()  
ax.set_ylabel(f'Counts', fontsize=18)
ax.set_xlabel("",fontsize=20)
        
for p in ax.patches:
        #ax.annotate('{:.0%}'.format(height), (p.get_x()+.15*width, p.get_y() + height + 0.01))
        ax.annotate("${}$".format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')
ax.set_xlabel(f'Number of active appliances', fontsize=18)     
ax.tick_params(axis='both', which='major', labelsize=18)
format_axes(ax)
ax.autoscale(tight=True)
savefig(fig_path+"active_appliances_distribution_plaid",format=".pdf")

In [45]:
for p in ax.patches:
    print(p)

Rectangle(xy=(0.85, 0), width=0.3, height=674, angle=0)
Rectangle(xy=(1.85, 0), width=0.3, height=413, angle=0)
Rectangle(xy=(2.85, 0), width=0.3, height=67, angle=0)


In [33]:
l, count=np.unique(np.hstack(labels), return_counts=True)
volgorde = np.argsort(count)
height = 0.8
fig = figure(fig_width=6)
plt.barh(np.array(l)[volgorde], count[volgorde], height=height, color="C1")
plt.tight_layout()
plt.tight_layout()
ax = plt.gca()  
ax.set_xlabel(f'Counts', fontsize=20)
#ax.set_ylabel(f'Number of active appliances', fontsize=20)
ax.set_xlabel("",fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.autoscale(tight=True)
savefig(fig_path+"label_distribution_plaid",format=".pdf")

In [34]:
unique_label = []
for l in labels:
    unique_label.append(l)

In [35]:
plaid_labels = []
for x in unique_label: 
    if x not in plaid_labels: 
        plaid_labels.append(x )
len(plaid_labels)

220

In [36]:
plaid_labels_counts = []
for l in plaid_labels:
    plaid_labels_counts.append(unique_label.count(l))

In [37]:
plaid_labels_counts[1]

2

In [38]:
plaid_labels[1]

['Fan', 'AC']

In [ ]:
single_app  = ['CFL']
double_app  = ["CFL", "LaptopCharger"]
single_lap  = ["LaptopCharger"]
single_kettle = ['Waterkettle']
cfls = []
cfls_laptop = []
laptops   = []
kettles   = []
for idx in range(len(labels)):
    x=list_intersection(single_app, labels[idx])
    if len(x)==1:
        cfls.append(idx)
        
    x=list_intersection(double_app, labels[idx])
    if len(x)==2:
        cfls_laptop.append(idx)
        
    x=list_intersection(single_lap, labels[idx])
    if len(x)==1:
        laptops.append(idx)   
        
    x=list_intersection(single_kettle, labels[idx])
    if len(x)==1:
        kettles.append(idx)   

In [ ]:
ones = []
for i, l in enumerate(labels[cfls]):
    if len(l)==1:
        ones.append(i)
ones

In [ ]:
current_cfl=generate_input_feature(current[cfls], voltage[cfls], "current", width=50,  p=2)

In [ ]:
decomposed_current_cfl=generate_input_feature(current[cfls], voltage[cfls], "decomposed_current", width=50,  p=2)

In [ ]:
x=decomposed_current_cfl[:,0,:]**2 + decomposed_current_cfl[:,1,:]**2
x.shape

In [ ]:
distance_cfl=generate_input_feature(current[cfls], voltage[cfls], "distance", width=50,  p=2)
vi_cfl=generate_input_feature(current[cfls], voltage[cfls], "vi", width=50,  p=2)

In [ ]:
decomposed_distance_cfl=generate_input_feature(current[cfls], voltage[cfls], "decomposed_distance", width=50,  p=2)
decomposed_vi_cfl=generate_input_feature(current[cfls], voltage[cfls], "decomposed_vi", width=50,  p=2)

In [ ]:
model  = TSNE(perplexity=20, min_grad_norm=1E-12, n_iter=10000, random_state=42)
TSNEF  = model.fit_transform(decomposed_current_cfl.reshape(len(decomposed_distance_cfl), -1))

In [ ]:
fig = figure(fig_width=4)
plt.plot(decomposed_current_cfl[1][0])
plt.tight_layout()
plt.ylabel("current $A$")
plt.xlabel("sample $w$")
#savefig(fig_path+"plaid_decomposed_currents_active", format=".pdf")

In [ ]:
fig = figure(fig_width=4)
plt.plot(decomposed_current_cfl[1][1])
plt.tight_layout()
plt.ylabel("current $A$")
plt.xlabel("sample $w$")
#savefig(fig_path+"plaid_decomposed_currents_non_active", format=".pdf")

In [ ]:
fig = figure(fig_width=4)
x=decomposed_current_cfl[1][1]**2 +decomposed_current_cfl[1][0]**2
x=x/x.max()

plt.plot(x)

In [ ]:
fig = figure(fig_width=4)
plt.plot(decomposed_current_cfl[1][1]+decomposed_current_cfl[1][0])
plt.tight_layout()
plt.ylabel("current $A$")
plt.xlabel("sample $w$")
savefig(fig_path+"plaid_decomposed_currents_non_active_plus_active", format=".pdf")

In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(vi_cfl[1][0], cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_vi_cfl", format=".pdf")

In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(distance_cfl[1][0], cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_distance_cfl", format=".pdf")

In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(decomposed_distance_cfl[1][0], cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_decomposed_distance_cfl_active", format=".pdf")


In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(decomposed_distance_cfl[1][1], cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_decomposed_distance_cfl_non_active", format=".pdf")

In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(decomposed_vi_cfl[1][1], cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_decomposed_vi_cfl_non_active", format=".pdf")

In [ ]:
fig = figure(fig_width=2.75, fig_height=2.75)
plt.imshow(decomposed_vi_cfl[1][0],cmap="Greys")
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_decomposed_vi_cfl_active", format=".pdf")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
mlb = MultiLabelBinarizer()
mlb.fit(labels)
y = mlb.transform(labels)

In [ ]:
PQ=calculatePower(current, voltage, NN=500)

In [ ]:
fig=figure(fig_width=7, fig_height=5)
plot_PQPlane(y, PQ, None, sample=False)
#savefig(fig_path+"plaid_PQ_plane",format=".pdf")